In [111]:
from collections import Counter, OrderedDict
import tensorflow as tf
import numpy as np
import tensorflow.keras
import functools
import math
import os 

%run midi_utils.ipynb

In [164]:
# Read all midi tokens into a single list of lists
filename = './training_data.txt'
with open(filename) as file:
    training_data = file.read().splitlines() 
    
# Split each midi by space to get its original token list form
training_data = [tokens.split(" ") for tokens in training_data]

# Remove all timesignature tokens because it turns out these are not really useful for anything
training_data = [[token for token in tokens if not token.startswith("time")] for tokens in training_data]

# Remove fraction subtraction craziness
replacements = [["0.084", "0.083"], ["0.167", "0.166"], ["0.334", "0.333"], ["0.332", "0.333"], ["0.667", "0.666"], ["wait:0.001", ""]]
for old, new in replacements:
    training_data = [[token.replace(old, new) for token in tokens] for tokens in training_data]
    
# Handle rare wait values by finding rarely occurring wait values and replacing them with the closest often-occurring value
tokens_combined = [item for sublist in training_data for item in sublist]
token_occurrences = Counter(tokens_combined)
occurrence_threshold = 100
need_to_be_replaced = sorted(list({float(k.split(":")[1]) for k, v in sorted(token_occurrences.items(), key=lambda item: item[1]) if k.startswith("wait") and v < occurrence_threshold}))
occurring_often = sorted(list({float(k.split(":")[1]) for k, v in sorted(token_occurrences.items(), key=lambda item: item[1]) if k.startswith("wait") and v >= occurrence_threshold}))

replacements = list()
for wait_time in need_to_be_replaced:
    replacements.append([str(wait_time), str(min(occurring_often, key=lambda x:abs(x-wait_time)))])
    
for old, new in replacements:
    training_data = [[token.replace(old, new) for token in tokens] for tokens in training_data]

In [165]:
tokens_combined = [item for sublist in training_data for item in sublist]
token_occurrences = Counter(tokens_combined)

len({k: v for k, v in sorted(token_occurrences.items(), key=lambda item: item[1])})

202